In [1]:
import pandas as pd

# Đọc dữ liệu từ file Excel
df = pd.read_excel('Content.xlsx')

# Loại bỏ các dòng có giá trị trống trong cột "Text"
df = df.dropna(subset=['Text'])

# Gộp các dòng liền kề có nhiều hơn 2 từ trong cột "Text"
merged_rows = []
current_text = ''
for i, row in df.iterrows():
    text = row['Text']
    if len(text.split()) > 2:
        current_text += ' ' + text
    else:
        if current_text:
            merged_rows.append({'STT': i, 'Text': current_text})
            current_text = text

# Xử lý dòng cuối cùng (nếu cần)
if current_text:
    merged_rows.append({'STT': i+1, 'Text': current_text})

# Tạo DataFrame mới từ các dòng đã gộp
merged_df = pd.concat([pd.DataFrame([row]) for row in merged_rows])

# Xuất kết quả vào file Excel mới
merged_df.to_excel('Merged Content.xlsx', index=False)

In [3]:
import pandas as pd

# Đọc dữ liệu từ file Excel
df = pd.read_excel('News Clean.xlsx')

# Gộp tất cả dòng nội dung của các giá trị của cột "Bank"
result = df.groupby('Bank')['Total'].apply(' '.join).reset_index()

# Lưu kết quả vào file Excel mới
result.to_excel('Merged News.xlsx', index=False)

In [15]:
import pandas as pd
import re
import string
from pyvi import ViTokenizer

def text_preprocessing(sent):
    sent = str(sent)

    with open('vietnamese.txt', 'r', encoding='utf-8') as f:
        stop_words = f.read().split('\n')

    sent = re.sub(f'[{string.punctuation}\d\n]', '', sent)
    sent = re.sub(r'[^\w\s]', '', sent)
    sent = ViTokenizer.tokenize(sent.lower())
    sent = [w for w in sent.split()]
    sent = [w for w in sent if w not in stop_words]
    return ' '.join(sent)

# Đường dẫn đến file dữ liệu
data_file = 'Merged News.xlsx'
key_file = 'Key.txt'
output_file = 'Processed News.xlsx'  

# Đọc dữ liệu từ file Excel
df = pd.read_excel(data_file)

# Xử lý dữ liệu văn bản trong cột 'Total' sử dụng hàm text_preprocessing
df['Total'] = df['Total'].apply(text_preprocessing)

# Đọc danh sách từ khóa từ file Key.txt & Xử lý văn bản tương tự
with open(key_file, 'r', encoding='utf-8') as file:
    keywords = [text_preprocessing(line.strip()) for line in file]

In [16]:
# Tạo cột mới trong DataFrame với tên từng từ khóa và tính tổng số lần xuất hiện
for keyword in keywords:
    df[keyword] = df['Total'].str.count(keyword)

# Ghi dữ liệu vào file Excel
df.to_excel(output_file, index=False)

In [17]:
import pandas as pd

# Đọc dữ liệu từ file Excel
df = pd.read_excel('Processed News.xlsx')

# Loại bỏ các cột chỉ có giá trị duy nhất là 0
df = df.loc[:, (df != 0).any()]

# Ghi dữ liệu vào file Excel
df.to_excel('Processed News.xlsx', index=False)

In [19]:
import pandas as pd
import numpy as np

# Fintech Score bằng cách nhân entropy với tần suất và lấy tổng
# Đọc dữ liệu từ file Excel
data = pd.read_excel('Processed News.xlsx')

# Lấy danh sách các từ khóa
keywords = data.columns[1:].tolist()

# Lấy dữ liệu ngân hàng và tần suất từ bảng
banks = {}
for _, row in data.iterrows():
    bank = row['Bank']
    frequencies = row[keywords].tolist()
    banks[bank] = frequencies

# Bước 1: Chuẩn hóa dữ liệu
keyword_values = {keyword: (min(frequencies), max(frequencies)) for keyword, frequencies in zip(keywords, zip(*banks.values()))}

normalized_data = {}
for bank, frequencies in banks.items():
    normalized_frequencies = {}
    for keyword, frequency in zip(keywords, frequencies):
        min_value, max_value = keyword_values[keyword]
        normalized_value = (frequency - min_value) / (max_value - min_value)
        normalized_frequencies[keyword] = normalized_value
    normalized_data[bank] = normalized_frequencies

# Bước 2: Xác định giá trị entropy của mỗi keyword
entropy_values = np.sum([[-(frequency * np.log2(frequency)) if frequency != 0 else 0 for frequency in frequencies.values()] for frequencies in normalized_data.values()], axis=0)
entropy = {keyword: entropy_values[index] for index, keyword in enumerate(normalized_data['ACB'].keys())}

# Bước 3: Tính Fintech Score cho mỗi ngân hàng
def fintech_score(entropy_values, frequencies):
    entropy_sum = np.sum(list(entropy_values.values()))
    fintech_scores = {}
    for bank, freq in frequencies.items():
        score = np.sum([entropy_values[keyword] * freq[keyword] for keyword in keywords]) / entropy_sum
        fintech_scores[bank] = score
    return fintech_scores

fintech_scores = fintech_score(entropy, normalized_data)

# In kết quả Fintech Score cho mỗi ngân hàng
for bank, score in fintech_scores.items():
    print(f"Ngân hàng {bank}: {score}")

# Tạo DataFrame từ kết quả Fintech Score
df = pd.DataFrame.from_dict(fintech_scores, orient='index', columns=['Fintech Score'])
df.index.name = 'Bank'

# Xuất kết quả ra file Excel
df.to_excel('Fintech Scores ME.xlsx')

Ngân hàng ​Vietcombank: 0.03985883230572404
Ngân hàng ACB: 0.4516419742665831
Ngân hàng Agribank: 0.40985058088722004
Ngân hàng Bac A Bank: 0.4252414038692805
Ngân hàng BaoViet Bank: 0.14315775023990937
Ngân hàng BIDV: 0.33306320326600625
Ngân hàng BVB: 0.6435631599293161
Ngân hàng Cathay United Bank: 0.033131029396516534
Ngân hàng CBB: 0.09603686250536955
Ngân hàng Eximbank: 0.5095057417813073
Ngân hàng HDBank: 0.32133158299967607
Ngân hàng KBank: 0.02019818344286887
Ngân hàng LienVietPostBank: 0.12188745561726431
Ngân hàng LPBank: 0.02157624775944663
Ngân hàng MB: 0.31881669966479054
Ngân hàng MSB: 0.3557292756677891
Ngân hàng Nam A Bank: 0.1174514574855882
Ngân hàng NCB: 0.3314826824258002
Ngân hàng Ngân hàng Nhà nước: 0.48310787605211897
Ngân hàng Ngân hàng Xăng dầu Petrolimex: 0.1214710329242106
Ngân hàng OCB: 0.40246347989221903
Ngân hàng PGBank: 0.1395268447293546
Ngân hàng Public Bank: 0.2345744060973768
Ngân hàng SCB: 0.4095597944648882
Ngân hàng SeABank: 0.08649779726831233
N

In [20]:
import pandas as pd
import numpy as np

# Fintech Score được tính dựa trên tỷ lệ giữa weighted entropy và tổng entropy
# Đọc dữ liệu từ file Excel
data = pd.read_excel('Processed News.xlsx')

# Lấy danh sách các từ khóa
keywords = data.columns[1:].tolist()

# Lấy dữ liệu ngân hàng và tần suất từ bảng
banks = {}
for _, row in data.iterrows():
    bank = row['Bank']
    frequencies = row[keywords].tolist()
    banks[bank] = frequencies

# Bước 1: Chuẩn hóa dữ liệu
keyword_values = {keyword: (min(frequencies), max(frequencies)) for keyword, frequencies in zip(keywords, zip(*banks.values()))}

normalized_data = {}
for bank, frequencies in banks.items():
    normalized_frequencies = {}
    for keyword, frequency in zip(keywords, frequencies):
        min_value, max_value = keyword_values[keyword]
        normalized_value = (frequency - min_value) / (max_value - min_value)
        normalized_frequencies[keyword] = normalized_value
    normalized_data[bank] = normalized_frequencies

# Bước 2: Xác định giá trị entropy của mỗi keyword
entropy_values = np.sum([[-(frequency * np.log2(frequency)) if frequency != 0 else 0 for frequency in frequencies.values()] for frequencies in normalized_data.values()], axis=0)
entropy = {keyword: entropy_values[index] for index, keyword in enumerate(normalized_data['ACB'].keys())}

# Bước 3: Tính Fintech Score cho mỗi ngân hàng
def fintech_score(entropy_values, frequencies):
    entropy_sum = np.sum(list(entropy_values.values()))
    fintech_scores = {}
    for bank, freq in frequencies.items():
        weighted_entropy = np.sum([entropy_values[keyword] * freq[keyword] for keyword in keywords])
        score = 1 - weighted_entropy / entropy_sum
        fintech_scores[bank] = score
    return fintech_scores

fintech_scores = fintech_score(entropy, normalized_data)

# In kết quả Fintech Score cho mỗi ngân hàng
for bank, score in fintech_scores.items():
    print(f"Ngân hàng {bank}: {score}")

# Tạo DataFrame từ kết quả Fintech Score
df = pd.DataFrame.from_dict(fintech_scores, orient='index', columns=['Fintech Score'])
df.index.name = 'Bank'

# Xuất kết quả ra file Excel
df.to_excel('Fintech Scores WE.xlsx')

Ngân hàng ​Vietcombank: 0.9601411676942759
Ngân hàng ACB: 0.5483580257334169
Ngân hàng Agribank: 0.59014941911278
Ngân hàng Bac A Bank: 0.5747585961307196
Ngân hàng BaoViet Bank: 0.8568422497600906
Ngân hàng BIDV: 0.6669367967339938
Ngân hàng BVB: 0.3564368400706839
Ngân hàng Cathay United Bank: 0.9668689706034834
Ngân hàng CBB: 0.9039631374946304
Ngân hàng Eximbank: 0.49049425821869275
Ngân hàng HDBank: 0.6786684170003239
Ngân hàng KBank: 0.9798018165571312
Ngân hàng LienVietPostBank: 0.8781125443827357
Ngân hàng LPBank: 0.9784237522405533
Ngân hàng MB: 0.6811833003352095
Ngân hàng MSB: 0.6442707243322109
Ngân hàng Nam A Bank: 0.8825485425144118
Ngân hàng NCB: 0.6685173175741999
Ngân hàng Ngân hàng Nhà nước: 0.516892123947881
Ngân hàng Ngân hàng Xăng dầu Petrolimex: 0.8785289670757894
Ngân hàng OCB: 0.5975365201077809
Ngân hàng PGBank: 0.8604731552706454
Ngân hàng Public Bank: 0.7654255939026232
Ngân hàng SCB: 0.5904402055351118
Ngân hàng SeABank: 0.9135022027316877
Ngân hàng SHB: 0.5